# Flights & Airports Exploratory SQL Notebook

This notebook demonstrates how to use **DuckDB** directly from Python to run fast, expressive SQL queries on the raw CSV ﬁles that you uploaded (`flights.csv`, `airports.csv`, and `lookupcodes.csv`).

All the queries are written in standard SQL, but you can mix and match with Python for visualization and further analysis.

> **Tip:** Change the file paths if you have moved the CSVs to a different folder.

In [ ]:
import duckdb, pandas as pd, matplotlib.pyplot as plt

# ➊ Register the three CSVs as DuckDB views so we can query them as tables
duckdb.sql("""CREATE OR REPLACE VIEW flights  AS SELECT * FROM read_csv_auto('/mnt/data/flights.csv');""")
duckdb.sql("""CREATE OR REPLACE VIEW airports AS SELECT * FROM read_csv_auto('/mnt/data/airports.csv');""")
duckdb.sql("""CREATE OR REPLACE VIEW lookup   AS SELECT * FROM read_csv_auto('/mnt/data/lookupcodes.csv');""")
print("Views created: flights (✈️), airports (🏙️), lookup (📋)")


## 1. Quick peek at the schema

In [ ]:
duckdb.sql("DESCRIBE flights").df()

In [ ]:
duckdb.sql("DESCRIBE airports").df()

In [ ]:
duckdb.sql("DESCRIBE lookup").df()

## 2. Top 10 busiest departure airports

In [ ]:
q1 = """
WITH departures AS (
    SELECT origin, COUNT(*) AS num_departures
    FROM flights
    GROUP BY origin
)
SELECT a.airport, a.city, a.state, d.num_departures
FROM departures d
JOIN airports a ON a.iata = d.origin
ORDER BY d.num_departures DESC
LIMIT 10;
"""
duckdb.sql(q1).df()


## 3. Airlines with the worst average arrival delay (≥15 min)

In [ ]:
q2 = """
WITH perf AS (
    SELECT carrier, AVG(arr_delay) AS avg_arr_delay_min
    FROM flights
    GROUP BY carrier
)
SELECT l.description   AS airline,
       ROUND(p.avg_arr_delay_min, 2) AS avg_arr_delay_min
FROM perf p
LEFT JOIN lookup l ON l.code = p.carrier
WHERE p.avg_arr_delay_min >= 15
ORDER BY p.avg_arr_delay_min DESC
LIMIT 15;
"""
duckdb.sql(q2).df()


## 4. 10 most‑flown routes

In [ ]:
q3 = """
SELECT origin, dest, COUNT(*) AS flights
FROM flights
GROUP BY origin, dest
ORDER BY flights DESC
LIMIT 10;
"""
duckdb.sql(q3).df()


## 5. Monthly flight volume trend

In [ ]:
q4 = """
SELECT month, COUNT(*) AS flights
FROM flights
GROUP BY month
ORDER BY month;
"""
df = duckdb.sql(q4).df()
df.plot(kind='bar', x='month', y='flights', figsize=(10,4), rot=0, title='Flights per month')
plt.ylabel('Number of flights')
plt.show()


## 6. Longest flights by great‑circle distance

In [ ]:
q5 = """
SELECT origin, dest, distance
FROM flights
ORDER BY distance DESC
LIMIT 10;
"""
duckdb.sql(q5).df()


## 7. Cancellation reasons breakdown

In [ ]:
q6 = """
SELECT l.description AS cancel_reason, COUNT(*) AS cancellations
FROM flights f
JOIN lookup l ON l.code = f.cancellation_code
WHERE f.cancelled = 1
GROUP BY l.description
ORDER BY cancellations DESC;
"""
duckdb.sql(q6).df()
